## Data Pre-processing

In [33]:
import pandas as pd
import scipy.io as sio

# Importing master excel document and extracting relevant data
data_raw = pd.read_excel('Input_Final_LIBSRATE.xlsx', 'Input', header = 4)
auto_data = data_raw.drop(list(range(0, 9)))
auto_data.drop(list(data.columns)[82:], axis = 1)
desired_columns = ['Parameters:', 'Population (low)', 'CpC (baseline)', 'S1.2', 'S1.3', 'S1.4', 'S1.5', 'S1.6', 'S1.7', 'S1.8', 'S1.9', 'S1.10', 'S1.11', 'S1.12', 'S1.13', 'S1.14']
auto_data = pd.concat([auto_data[col] for col in auto_data.columns if col in desired_columns], axis = 1)
auto_data.columns = ['Year', 'Population', 'CpC', 'Closures', 'Bumpers', 'Engine Blocks', 'Heat Exchangers', 'Cylinder Heads', 'Suspension', 'Steering', 'Wheels', 'Transmission/Driveline', 'Brake Componenets', 'Other Engine', 'Body', 'Other Components']
auto_data.set_index('Year')
writer = pd.ExcelWriter('LIBS rate Input Data.xlsx')
auto_data.to_excel(writer, 'Auto Data', index = False, index_label = 'Year')

# Importing alloy information
alloys_raw = sio.loadmat('alloys_info.mat')

# Parsing .MAT raw data into hierarchical array representation
# [alloy, type, [nominal[], min[], max[]]]
alloys_info = []
for alloy in alloys_raw['alloys_info'][0]:
    alloys_info.append([alloy[0][0][0][0], alloy[3][0], 'Nominal', *list(alloy[1][0])])
    alloys_info.append([alloy[0][0][0][0], alloy[3][0], 'Minimum', *list(alloy[2][1])])
    alloys_info.append([alloy[0][0][0][0], alloy[3][0], 'Maximum', *list(alloy[2][0])])
alloys_info = pd.DataFrame(alloys_info)
alloys_info.columns = ['Alloy', 'Type', 'Level', 'Si', 'Fe', 'Cu', 'Mn', 'Mg', 'Cr', 'Ni', 'Zn', 'Ti']
alloys_info_table = pd.pivot_table(alloys_info, 
                       index = ['Alloy', 'Type', 'Level'],
                       values = ['Level', 'Si', 'Fe', 'Cu', 'Mn', 'Mg', 'Cr', 'Ni', 'Zn', 'Ti'])
alloys_info_table.to_excel(writer, 'Alloy Data')
alloys_info.set_index(['Alloy', 'Type', 'Level'], drop = True)

writer.save()

AttributeError: 'collections.OrderedDict' object has no attribute 'columns'

## Set Scenario Conditions and Parameters

In [31]:
##### SCENARIO CONDITIONS #####
# Population: 1=baseline;
PQ = 1 

# Cars per capita: 1=baseline
CpCQ = 1

# Dismantling before ELVyear: 0=none, 1=low, 2=high, 3=all
ELVyear = 1990
ELVQ1 = 0

# Dismantling from ELVyear: 0=none, 1=low, 2=high, 3=all
ELVQ2 = 1

# Alloy sorting scenario: 0=none, 2=laser sorting
ZQ = 2

# LIBS implmentation rate: 0=none, 1 = low, 2 = medium, 3 = immediate, 100%
LIBSrate = 1

# Lifetime scenario: 2=baseline, 1=low, 3=high
LTQ = 2

# Demagging: 1=on, 2=off
DMGQ = 1

Zorba_export = 0

##### PARAMETERS #####
# Number of alloy groups that are sorted, e.g. 1xxx, 3xxx
NAG = 8

# This parameter reduces the amount of scrap that can be intelligently sorted as empirical data 
# has been collected that indicates that approximately 25% of scrap partiuculates are under 1 inch
sizelimited = 0.25

##### INPUT VARIABLES #####
# importing all data from excel sheet: 'LIBS Rate Input Data.xlsx'
data = pd.read_excel('LIBS Rate Input Data.xlsx', sheet_name = None)
old_scrap = data['Old Scrap']
old_scrap_comp = data['Old Alloy Compositions']
laser_sorting_alloy_groups = data['Laser Sorting Alloy Groups']
alloy_data = data['Alloy Data']
auto_data = data['Auto Data']

alloy_names = list(other_old_scrap_comp.index)
element_names = list(data['Old Alloy Compositions'].columns)
raw_material_names = list(data['Laser Sorting Alloy Groups'].index)
comp_group_names = list(data['Auto Data'].columns)[3:]
years = list(data['Auto Data'].index)

In [32]:
years

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
